In [6]:
import cv2
import os

labels = []
def load_images_from_folder(folder):
    data = []   
    for filename in os.listdir(folder):
        if filename.lower().endswith('.jpg'):
            img = cv2.imread(os.path.join(folder, filename))
            data.append(img)

            # for printing results
            labels.append(filename)
    return data

dog_images = load_images_from_folder('training/dog')
cat_images = load_images_from_folder('training/cat')

In [8]:
import numpy as np

dog_images = np.array(dog_images)
cat_images = np.array(cat_images)

print(dog_images.shape)
print(cat_images.shape)

(10, 224, 224, 3)
(10, 224, 224, 3)


In [9]:
# use it as a magic tool
from transformers import CLIPProcessor, CLIPModel
import torch

clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

# CLIP
def magic_function(images):
    inputs = clip_processor(images=images, return_tensors="pt", padding=True)
    with torch.no_grad():
        features = clip_model.get_image_features(**inputs)
    return features.numpy()

dog_images_features = magic_function(list(dog_images))
cat_images_features = magic_function(list(cat_images))

/Users/quangvinh/miniconda3/envs/torchenv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
print(dog_images_features.shape)
print(cat_images_features.shape)

(10, 512)
(10, 512)


In [11]:
merged_features = np.concatenate((dog_images_features, cat_images_features), axis=0)
print(merged_features.shape)

(20, 512)


In [12]:

from sklearn.cluster import KMeans

# KMeans
kmeans = KMeans(n_clusters=2, random_state=0).fit(merged_features)
print(kmeans.labels_)

for real_label, predicted_label in zip(labels, kmeans.labels_):
    print(f"Cluster {predicted_label}: {real_label}")

[1 1 1 1 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
Cluster 1: dog.1.jpg
Cluster 1: dog.0.jpg
Cluster 1: dog.2.jpg
Cluster 1: dog.3.jpg
Cluster 0: dog.7.jpg
Cluster 1: dog.6.jpg
Cluster 1: dog.4.jpg
Cluster 1: dog.5.jpg
Cluster 1: dog.8.jpg
Cluster 1: dog.9.jpg
Cluster 0: cat.6.jpg
Cluster 0: cat.7.jpg
Cluster 0: cat.5.jpg
Cluster 0: cat.4.jpg
Cluster 0: cat.0.jpg
Cluster 0: cat.1.jpg
Cluster 0: cat.3.jpg
Cluster 0: cat.2.jpg
Cluster 0: cat.9.jpg
Cluster 0: cat.8.jpg
